In [1]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [9]:
import keras
import keras_nlp

In [22]:
import json
data = []
with open('/kaggle/input/databricks-dolly-15k/databricks-dolly-15k.jsonl') as file:
    for line in file:
        features = json.loads(line)
        if features["context"]:
            continue
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

data = data[:2000]

In [5]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

## Inference before fine tuning


In [10]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
1. Take a trip to Europe.
2. Take a trip to Europe.
3. Take a trip to Europe.
4. Take a trip to Europe.
5. Take a trip to Europe.
6. Take a trip to Europe.
7. Take a trip to Europe.
8. Take a trip to Europe.
9. Take a trip to Europe.
10. Take a trip to Europe.
11. Take a trip to Europe.
12. Take a trip to Europe.
13. Take a trip to Europe.
14. Take a trip to Europe.
15. Take a trip to Europe.
16. Take a trip to Europe.
17. Take a trip to Europe.
18. Take a trip to Europe.
19. Take a trip to Europe.
20. Take a trip to Europe.
21. Take a trip to Europe.
22. Take a trip to Europe.
23. Take a trip to Europe.
24. Take a trip to Europe.
25. Take a trip to


In [7]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is the process by which plants use the energy from the sun to convert water and carbon dioxide into oxygen and glucose. The process begins with the absorption of light energy by chlorophyll molecules in the leaves of plants. The energy from the light is used to split water molecules into hydrogen and oxygen. The oxygen is released into the atmosphere, while the hydrogen is used to make glucose. The glucose is then used by the plant to make energy and grow.

Explanation:
Photosynthesis is the process by which plants use the energy from the sun to convert water and carbon dioxide into oxygen and glucose. The process begins with the absorption of light energy by chlorophyll molecules in the leaves of plants. The energy from the light is used to split water molecules into hydrogen and oxygen. The oxygen is released into the atmosphere, while the hydrogen is used to make gluc

In [8]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [20]:
gemma_lm.preprocessor.sequence_length = 512
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=2, batch_size=1)

Epoch 1/2
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1409s 701ms/step - loss: 0.4331 - sparse_categorical_accuracy: 0.5439
Epoch 2/2
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1402s 698ms/step - loss: 0.4218 - sparse_categorical_accuracy: 0.5531


In [24]:
gemma_lm.save("/kaggle/working/gemma_lm_model.keras")

In [2]:
# !pip install python-doctr[tf] 
# !pip install tf2onnx 

# !pip install tensorflow==2.15.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 70.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.9/304.9 kB 19.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=0d03059f837139447780e002898cae0c3656bf8fbc91032d9a8f791d47ef0046
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [13]:
!pip install pytesseract

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [14]:
!pip install Pillow
from PIL import Image 
import pytesseract

In [15]:
def ocr_pytesseract(image_path):
    image = Image.open(image_path)  # Open the image
    text = pytesseract.image_to_string(image)  # Extract text
    return text



In [26]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
Europe is a great place to visit for a variety of reasons. It is a continent that is rich in history, culture, and natural beauty. There are many different countries to explore, each with its own unique culture and history. Some of the most popular destinations in Europe include France, Italy, Germany, Spain, and the United Kingdom. Each of these countries has its own unique attractions, from the Eiffel Tower in Paris to the Colosseum in Rome. There are also many natural wonders to be found in Europe, such as the Alps in Switzerland and the Grand Canyon in Italy. No matter what your interests are, there is sure to be something in Europe that will appeal to you.


In [16]:
image_path = '/kaggle/input/testpresimage/image.png'
presText = ocr_pytesseract(image_path)
print(presText)

Pr. A. Sharma, New Delhi Clinic, +91 9876543210

Today, a middle-aged man, Kumar, came in complaining of dizziness and fatigue. He’s been
having issues with high blood pressure for a while, and after a quick check-up, it turns out he
also has blood sugar levels that aren't quite right. We've talked about starting him on some
medication to keep both conditions under control.

For his heart, he'll need one pill in the morning — something like Amlodipine, 5 mg. For his
sugar, it’s a twice-daily thing after meals — maybe Metformin, 500 mg. And at night, there’s
that statin — Atorvastatin, 10 mg should do the trick.

| reminded him about cutting down on salt, sticking to whole grains, and squeezing in at least
half an hour of light exercise each day. He’s supposed to check back in with me in a month,
but if things go south, he knows to call sooner.

— A. Sharma (signed)
20/10/2024



In [28]:
presTemp = """
Extract and list the following information from the doctor's prescription slip:
1. Patient Name
2. Patient Age
3. Patient Gender
4. Date of Prescription
5. Diagnosis
6. Medications (with dosage and frequency)
7. Additional instructions
8. Doctor's Name

Prescription Text:
{presText}
"""

prestFormattedTem = presTemp.format(presText=presText)
print(prestFormattedTem) 


Extract and list the following information from the doctor's prescription slip:
1. Patient Name
2. Patient Age
3. Patient Gender
4. Date of Prescription
5. Diagnosis
6. Medications (with dosage and frequency)
7. Additional instructions
8. Doctor's Name

Prescription Text:
Pr. A. Sharma, New Delhi Clinic, +91 9876543210

Today, a middle-aged man, Kumar, came in complaining of dizziness and fatigue. He’s been
having issues with high blood pressure for a while, and after a quick check-up, it turns out he
also has blood sugar levels that aren't quite right. We've talked about starting him on some
medication to keep both conditions under control.

For his heart, he'll need one pill in the morning — something like Amlodipine, 5 mg. For his
sugar, it’s a twice-daily thing after meals — maybe Metformin, 500 mg. And at night, there’s
that statin — Atorvastatin, 10 mg should do the trick.

| reminded him about cutting down on salt, sticking to whole grains, and squeezing in at least
half an hou

In [33]:
prompt = template.format(
    instruction=prestFormattedTem,
    response="", )

# print(prompt)


generated_text = gemma_lm.generate(prompt)

print(generated_text)

Instruction:

Extract and list the following information from the doctor's prescription slip:
1. Patient Name
2. Patient Age
3. Patient Gender
4. Date of Prescription
5. Diagnosis
6. Medications (with dosage and frequency)
7. Additional instructions
8. Doctor's Name

Prescription Text:
Pr. A. Sharma, New Delhi Clinic, +91 9876543210

Today, a middle-aged man, Kumar, came in complaining of dizziness and fatigue. He’s been
having issues with high blood pressure for a while, and after a quick check-up, it turns out he
also has blood sugar levels that aren't quite right. We've talked about starting him on some
medication to keep both conditions under control.

For his heart, he'll need one pill in the morning — something like Amlodipine, 5 mg. For his
sugar, it’s a twice-daily thing after meals — maybe Metformin, 500 mg. And at night, there’s
that statin — Atorvastatin, 10 mg should do the trick.

| reminded him about cutting down on salt, sticking to whole grains, and squeezing in at leas

In [50]:
import csv
import os

response_part = generated_text.split("Response:")[1].strip()

lines = response_part.split("\n")

data = {
    "Patient Name": "",
    "Patient Age": "",
    "Patient Gender": "",
    "Date of Prescription": "",
    "Diagnosis": "",
    "Medications": "",
    "Additional Instructions": "",
    "Doctor's Name": ""
}

medications = []
additional_instructions = []

for line in lines:
    if line.startswith("Patient Name:"):
        data["Patient Name"] = line.split(": ", 1)[1]
    elif line.startswith("Patient Age:"):
        data["Patient Age"] = line.split(": ", 1)[1]
    elif line.startswith("Patient Gender:"):
        data["Patient Gender"] = line.split(": ", 1)[1]
    elif line.startswith("Date of Prescription:"):
        data["Date of Prescription"] = line.split(": ", 1)[1]
    elif line.startswith("Diagnosis:"):
        data["Diagnosis"] = line.split(": ", 1)[1]
    elif line.startswith("Medications:"):
        continue  # Skip "Medications" label
    elif line.startswith("Additional Instructions:"):
        continue  # Skip "Additional Instructions" label
    elif line.startswith("Doctor's Name:"):
        data["Doctor's Name"] = line.split(": ", 1)[1]
    elif line.startswith("1.") or line.startswith("2.") or line.startswith("3.") and "mg" in line:
        medications.append(line.strip())
    elif line.startswith("1.") or line.startswith("2.") or line.startswith("3."):
        additional_instructions.append(line.strip())

data["Medications"] = "; ".join(medications)
data["Additional Instructions"] = "; ".join(additional_instructions)

file_exists = os.path.isfile('prescription_extracted.csv')


print(data)

{'Patient Name': 'Kumar', 'Patient Age': '45', 'Patient Gender': 'Male', 'Date of Prescription': '20/10/2024', 'Diagnosis': 'High Blood Pressure, Diabetes', 'Medications': '1. Amlodipine, 5 mg; 2. Metformin, 500 mg; 3. Atorvastatin, 10 mg; 1. Cut down on salt; 2. Stick to whole grains', 'Additional Instructions': '3. Exercise at least half an hour each day', "Doctor's Name": 'A. Sharma'}


In [51]:
with open('prescription_extracted.csv', mode='a', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=data.keys())
    
    if not file_exists:
        writer.writeheader()
    
    writer.writerow(data)

print("Data appended to 'prescription_extracted.csv'")

Data appended to 'prescription_extracted.csv'
